필요한 모듈 import

In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
import numpy as np
import pandas as pd

In [4]:
# !pip install selenium

     |████████████████████████████████| 904 kB 1.8 MB/s eta 0:00:01


In [6]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [7]:
from selenium.webdriver.chrome.options import Options

크롤링 데이터 저장을 위한 데이터프레임 생성

In [123]:
kopis_address = pd.DataFrame(columns=[
    '제작사', '주소', '장르'
])

tmp_kopis_address = pd.DataFrame(columns=[
    '제작사', '주소', '장르'
])


데이터프레임 형태 확인

In [124]:
tmp_kopis_address

,제작사,주소,장르


크롤링 시행

In [133]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")

driver = webdriver.Chrome('/Users/hyungyeonghong/Desktop/chromedriver', options = chrome_options )
driver.maximize_window()

driver.implicitly_wait(time_to_wait = 5)

for page_index in range(0, 18):
    print(page_index+1)
    
    # 무용, 국악, 서울 체크박스 선택: 첫 페이지에서만 실행
    if page_index == 0:
        driver.get(url = f'https://www.kopis.or.kr/por/db/mnfct/mnfct.do?menuId=MNU_00022&searchWord=&searchType=total#p{page_index+1}#01#ASC#########1#0#')
        time.sleep(5)

        tmp = driver.find_element_by_xpath('//*[@id="su_con"]/div[1]/div[2]/p/a')
        tmp.click()
        time.sleep(2)

        tmp = driver.find_element_by_id("srchMt2zGenreCode_5")
        tmp.click()
        time.sleep(2)

        tmp = driver.find_element_by_id("srchMt2zGenreCode_6")
        tmp.click()
        time.sleep(2)

        tmp = driver.find_element_by_id("srchSignguCode_1")
        tmp.click()
        time.sleep(2)

        tmp = driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/div[3]/div/div/a[1]')
        tmp.click()
        time.sleep(2)
    
    if page_index < 10:
        tmp = driver.find_element_by_xpath('//*[@id="su_con"]/div[6]/div/a[13]')
        tmp.click()
        time.sleep(2)
        driver.implicitly_wait(time_to_wait = 5)
        
        continue
    
    if page_index == 10:
        tmp = driver.find_element_by_xpath('//*[@id="su_con"]/div[6]/div/a[11]')
        tmp.click()
        time.sleep(2)
        driver.implicitly_wait(time_to_wait = 5)
        
        continue
        
    
    # 마지막 페이지 극단 수 다름
    if page_index !=17:
        num_of_places_in_page = 30
        
    elif page_index == 17:
        num_of_places_in_page = 9
        
        
    # 극단 크롤링
    for j in range(0, num_of_places_in_page):
        
        driver.implicitly_wait(time_to_wait = 5)
        
        tmp = driver.find_element_by_xpath(f'//*[@id="table_reaction"]/table/tbody/tr[{j+1}]/td[2]/a')
        tmp.click()
        time.sleep(3)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
               
        plc = soup.select_one('#su_con > div.vw_lsbk > ul.ro_utb > li:nth-of-type(1) > div:nth-of-type(1) > dl > dd').get_text().replace('\n','')
        address = soup.select_one('#su_con > div.vw_lsbk > ul.ro_utb > li:nth-of-type(3) > div.sgl > dl > dd').get_text().replace('\n','')
        genre = soup.select_one('#su_con > div.vw_lsbk > ul.ro_utb > li:nth-of-type(4) > div:nth-of-type(2) > dl > dd').get_text().replace('\n','')
        
        tmp_kopis_address = tmp_kopis_address.append({'제작사' : plc, '주소' : address, '장르' : genre}, ignore_index=True)
        time.sleep(2)
        
        driver.back()
        time.sleep(2)
        
    # 다음 페이지로 넘어가기
    if page_index != 17:
        tmp = driver.find_element_by_xpath('//*[@id="su_con"]/div[6]/div/a[11]')
        tmp.click()
        time.sleep(2)
        driver.implicitly_wait(time_to_wait = 5)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


tmp_kopis_address 데이터 kopis_address로 옮겨오기

In [136]:
kopis_address = tmp_kopis_address

In [137]:
kopis_address

,제작사,주소,장르
0,(사)가야금병창보존회,서울특별시 서초구 반포대로14길 71 서초엘지에클라트오피스텔1929호,국악
1,(사)가야금산조진흥회,서울특별시 서초구 서초중앙로 15,국악
2,(사)경기국극진흥회,서울특별시 종로구 종로 313,국악
3,(사)경서도창악회,서울특별시 서초구 사임당로18길 52-2,국악
4,(사)공명,"서울특별시 종로구 동숭4가길2, 2층(동숭동)",국악
...,...,...,...
514,황병기 작품 보존회,서울특별시 서초구,국악
515,후(后)Korean Dance Company,서울특별시 중랑구 동일로157길 13(묵동),무용
516,후댄스컴퍼니,서울특별시 종로구,무용
517,휴먼 스탕스,서울특별시 종로구,무용


데이터프레임을 csv 파일로 변환

In [140]:
kopis_address.to_csv("kopis_address.csv", index = False, encoding = "utf-8")